In [1]:
from analysis import *
import matplotlib.pyplot as plt
import pickle

In [2]:
nodes = []

for filename in os.listdir(os.fsencode('dumps')):
    filename = filename.decode("utf-8")
    with open(f'dumps/{filename}', 'rb') as f:
        nodes.append(pickle.load(f))
        
print(f'Loaded {len(nodes)} nodes.')

Loaded 16 nodes.


In [6]:
blocks = get_all_blocks(nodes)
for id, block in blocks.items(): 
    print(block.miner)

MINER_US_5
MINER_US_7
MINER_US_5
MINER_US_5
MINER_US_6
MINER_US_2
MINER_US_2
MINER_US_7
MINER_US_4
MINER_US_5
MINER_US_7
MINER_US_4
MINER_US_1
MINER_US_1
MINER_US_6
MINER_US_7
MINER_US_3
MINER_CH_0
MINER_CH_0
MINER_CH_0
MINER_CH_0
MINER_CH_0
MINER_CH_0
MINER_RU_0
MINER_CH_0
MINER_CH_0
MINER_CH_0
MINER_CH_0
MINER_RU_0
MINER_CH_0
MINER_CH_0
MINER_CH_0
MINER_CH_0
MINER_CH_0
MINER_CH_0
MINER_CH_0
